In [ ]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets

In [ ]:
#one hot encoding
import numpy as np
def one_hot(ys,num_classes):
  one=np.zeros((ys.shape[0],num_classes))
  for i, y in enumerate(ys):
    one[i][y]=1
  return one

In [ ]:
#show data
import matplotlib.pyplot as plt
def show(image,nb_imgs):
  fig, axes = plt.subplots(nb_imgs, nb_imgs, figsize=(15, 15))
  for i in range(nb_imgs):
      for j in range(nb_imgs):
          axes[i][j].imshow(image[i*j].reshape(28, 28),cmap='gray')

In [ ]:
#load data 
from tensorflow.keras.datasets import mnist 
def load_data():
  (x_train_raw, y_train_raw), (x_val_raw, y_val_raw) = mnist.load_data()
  num_classes=len(list(set(y_train_raw)))

  x_train=x_train_raw.astype("float32")/255
  x_val=x_val_raw.astype("float32")/255

  y_train=one_hot(y_train_raw,num_classes)
  y_val=one_hot(y_val_raw,num_classes)
  return (x_train,y_train),(x_val,y_val)

In [ ]:
#flat data
def flat(data):
  data_flat=[]
  for i in data:
    data_flat.append(i.flatten())
  data_flat=np.array(data_flat)
  return data_flat

In [ ]:
(x_train,y_train),(x_val,y_val)=load_data()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.1)

In [ ]:
x_train_flat=flat(x_train)
x_test_flat=flat(x_test)
x_val_flat=flat(x_val)

In [ ]:
print("Shape of images in val dataset {}".format(x_val_flat.shape))
print("Shape of classes in val dataset {}".format(y_val.shape))
print("Shape of images in training dataset {}".format(x_train_flat.shape))
print("Shape of classes in training dataset {}".format(y_train.shape))
print("Shape of images in testing dataset {}".format(x_test_flat.shape))
print("Shape of classes in testing dataset {}".format(y_test.shape))

In [ ]:
show(x_test,4)

In [ ]:
#placeholder
tf.compat.v1.disable_eager_execution()
x=tf.compat.v1.placeholder(tf.float32,[None,784]) 
y=tf.compat.v1.placeholder(tf.float32,[None,10]) 

In [ ]:
#variable
w=tf.Variable(tf.ones([784,10]))
b=tf.cast(tf.Variable(np.random.random(10)),tf.float32)

In [ ]:
logit=tf.matmul(x,w)+b
pred=tf.nn.softmax(logits=logit)

In [ ]:
#calculating loss of our model
entropy=tf.nn.softmax_cross_entropy_with_logits(logits=logit,labels=y)
loss = tf.reduce_mean(entropy)

In [ ]:
#calculating accuracy of our model 
correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# Optimizer
learning_rate = 0.01
batch_size = 128
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
optimizer1=tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate,
    beta1=0.9,
    beta2=0.999,
    epsilon=1e-08,
    use_locking=False,
    name='Adam').minimize(loss)

In [ ]:
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

In [ ]:
# when using Adam
def set_lr(epoch,learning_rate):
  if epoch>300:
    if epoch%100==0:
      learning_rate=learning_rate/10
  return learning_rate

In [ ]:
#training
epochs = 1000
train_loss,test_loss,num_epoch,acc=[],[],[],[]
for epoch in range(epochs):
  learning_rate=set_lr(epoch,learning_rate)  # when using Adam
  _,a1=sess.run([optimizer1,loss],feed_dict={x:x_train_flat,y:y_train})
  a2,a3=sess.run([loss,accuracy],feed_dict={x:x_val_flat,y:y_val})
  train_loss.append(a1)
  test_loss.append(a2)
  acc.append(a3)
  num_epoch.append(epoch)
  if epoch%100==0:
    print(learning_rate)
    print("Epochs {} val_loss = {} and train loss = {} and accuracy = {}".format(epoch, a2,a1,a3))

In [ ]:
# loss and accuracy GradientDescentOptimizer
%matplotlib inline
plt.figure(figsize=(7, 5))
plt.title('GradientDescentOptimizer')
plt.plot(range(epochs), train_loss,label="rain Loss")
plt.plot(range(epochs),test_loss,label="test Loss")
plt.legend()
plt.show()
plt.figure(figsize=(7, 5))
plt.plot(range(epochs),acc,label="acc")
plt.legend()
plt.show()

In [ ]:
# loss and accuracy AdamOptimizer
%matplotlib inline
plt.figure(figsize=(7, 5))
plt.title('AdamOptimizer')
plt.plot(range(epochs), train_loss,label="rain Loss")
plt.plot(range(epochs),test_loss,label="test Loss")
plt.legend()
plt.show()
plt.figure(figsize=(7, 5))
plt.plot(range(epochs),acc,label="acc")
plt.legend()
plt.show()

In [ ]:
#evaluate in val dataset
val_loss, val_acc=sess.run([loss,accuracy],feed_dict={x:x_val_flat,y:y_val})
print("loss and Accuracy in the validation dataset: {} and {}%".format(val_loss, val_acc*100))
#evaluate in test dataset
test_loss, test_acc=sess.run([loss,accuracy],feed_dict={x:x_test_flat,y:y_test})
print("loss and Accuracy in the testting dataset: {} and {}%".format(test_loss, test_acc*100))

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = sess.run(pred, feed_dict={x : x_val_flat})
true_class=np.argmax(y_val,1)
predicted_class=np.argmax(y_pred,1)
cm=confusion_matrix(predicted_class,true_class)

In [ ]:
#confusion matrix GradientDescentOptimizer
import seaborn as sns
%matplotlib inline
plt.figure(figsize=(10,7))
plt.title('Confusion Matrix')
ax=sns.heatmap(cm, annot = True,annot_kws={'size':10},linewidths=1,cmap=plt.cm.Reds)
ax.set(xlabel="Predicted label", ylabel="True label")
ax.xaxis.tick_top()

In [ ]:
#confusion matrix AdamOptimizer
import seaborn as sns
%matplotlib inline
plt.figure(figsize=(10,7))
plt.title('Confusion Matrix')
ax=sns.heatmap(cm, annot = True,annot_kws={'size':10},linewidths=1,cmap=plt.cm.Reds)
ax.set(xlabel="Predicted label", ylabel="True label")
ax.xaxis.tick_top()

In [ ]:
#Finding error outputs
y_pred = sess.run(pred, feed_dict={x : x_val_flat})
idx=np.argmax(y_pred,1)==np.argmax(y_val,1) 
cmp=np.where(idx==False) #indices of error outputs
# plotting errors
fig, axes = plt.subplots(10, 5, figsize=(10,10))
fig.subplots_adjust(hspace=0.3, wspace=0.3)
cls_true=np.argmax(y_val,1)[cmp]
cls_pred=np.argmax(y_pred,1)[cmp]
images=x_test[cmp]
print(images.shape)
for i, ax in enumerate(axes.flat):
        ax.imshow(images[i].reshape(28,28), cmap='gray')
        xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])
        ax.set_xlabel(xlabel)
        ax.set_xticks([])
        ax.set_yticks([])      
plt.show()

In [ ]:
weight_grad=["GradientDescentOptimizer",epochs,batch_size,val_loss, val_acc,test_loss, test_acc,sess.run(w),sess.run(b)]
import pickle
with open("GradientDescentOptimizer.bin", "wb") as output:
    pickle.dump(weight_grad, output)
with open("GradientDescentOptimizer.bin", "rb") as data:
    weight_grad = pickle.load(data)

In [ ]:
weight_adam=["AdamOptimizer",epochs,batch_size,val_loss, val_acc,test_loss, test_acc,sess.run(w),sess.run(b)]
import pickle
with open("AdamOptimizer.bin", "wb") as output:
    pickle.dump(weight_adam, output)
with open("AdamOptimizer.bin", "rb") as data:
    weight_adam = pickle.load(data)

In [ ]:
from prettytable import PrettyTable
    
p = PrettyTable()
p.field_names = ["Optimizer", "Epoch", "Batch_size","loss_val","Accuracy_val","loss_test","accuracy_test"]
p.add_row(weight_grad[0:7])
p.add_row(weight_adam[0:7])

print(p)

In [ ]:
+--------------------------+-------+------------+------------+--------------+------------+---------------+
|        Optimizer         | Epoch | Batch_size |  loss_val  | Accuracy_val | loss_test  | accuracy_test |
+--------------------------+-------+------------+------------+--------------+------------+---------------+
| GradientDescentOptimizer |  1000 |    128     | 0.5862813  |    0.8697    | 0.6169034  |     0.8595    |
|      AdamOptimizer       |  1000 |    128     | 0.28068522 |    0.9277    | 0.27760762 |     0.9245    |
+--------------------------+-------+------------+------------+--------------+------------+---------------+

In [ ]:
import matplotlib.pyplot as plt
def show_result(X_true, y_true):
    plt.imshow(X_true.reshape(28, 28))
    print(np.argmax(y_true))
    x_true_flat=X_true.flatten()
    print(sess.run(accuracy,feed_dict={x:x_true_flat.reshape(1,784),y:y_true.reshape(1,10)}))
    y_preds = sess.run(pred, feed_dict={x : [x_true_flat]})
    print(np.argmax(y_preds))
    plt.colorbar()
show_result(x_test[0], y_test[0])